---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [38]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
pd.set_option('display.max_columns',50)

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

In [44]:
def blight_model(fprefix = ''):
    fnames = ['train','test','addresses','latlons']
    data = [pd.read_csv(fprefix+fname+'.csv', encoding = "ISO-8859-1") for fname in fnames]
    train_data, test_data, addresses, latlons = data
        
    addresses = addresses.set_index('address').join(latlons.set_index('address'), how='left')
    train_data = train_data.set_index('ticket_id').join(addresses.set_index('ticket_id'))
    test_data = test_data.set_index('ticket_id').join(addresses.set_index('ticket_id'))
    
    #Remove columns from train data that do not appear in test data.
    train_cols_to_drop = [col for col in train_data.columns if col not in test_data.columns and col!='compliance']
    train_data.drop(train_cols_to_drop, axis=1,inplace=True)
    
    # Remove all training data with no compliance information.
    train_data.dropna(subset=['compliance'],inplace=True)
    
    # We can save on memory by converting the columns with few different values from floats to categories. 
    cols_to_cat = []
    for col in train_data.columns:
        if train_data[col].nunique() < 100 and col!='compliance':
            cols_to_cat.append(col)

    for col in cols_to_cat:
        for df in train_data, test_data:
            df[col] = df[col].astype('category')
            
    # Remove non-USA data
    train_data=train_data[train_data['country']=='USA']
    
    # All of the location information will be assumed to be given by the latitude and longitude, so we can eliminate 
    # some of the columns that are redundant or have no meaningful information.
    redundant_info = ['mailing_address_str_number', 'mailing_address_str_name','violation_street_number',\
                  'violation_street_name','violation_zip_code','non_us_str_code','city','state', 'zip_code', 'country']
    # We should also remove any personal information. It won't give us meaningful information anyway. 
    personal_info = ['violator_name','inspector_name']
    # We will also drop 'violation_description' since it is too varied to contain useful info for modelling.
    other_to_drop = ['violation_description', 'grafitti_status','admin_fee', 'state_fee', 'clean_up_cost']
    cols_to_remove = redundant_info + personal_info + other_to_drop
    for df in train_data, test_data:
        df.drop(cols_to_remove, axis=1,inplace=True)
        
    # We can get rid of the train data points that have no lat/long info, but for the test points with missing lat/long 
    # we should fill it in with the mean values of the lat and lon. 
    train_data.dropna(subset=['lat','lon'],inplace = True) 
    test_data['lat'].fillna(test_data.lat.mean(),inplace = True)
    test_data['lon'].fillna(test_data.lon.mean(),inplace = True)
    # We will also get rid of the train data points that have no hearing_date
    train_data.dropna(subset=['hearing_date'],inplace = True) 
    
    # We will replace 'ticket_issue_date' and 'hearing-date' with a new feature that we will call 'time_gap'.
    # First convert data to datetime data.
    for df in train_data, test_data:
        for date in ['hearing_date','ticket_issued_date']:
            df[date] = pd.to_datetime(df[date])
        df['time_gap'] = df['hearing_date'].subtract(df['ticket_issued_date'])
        df['time_gap'] = df['time_gap'].dt.days
        df.drop(['hearing_date','ticket_issued_date'], axis=1, inplace = True)
    test_data['time_gap'].fillna(test_data.time_gap.mean(),inplace = True)
    
    # Turn compliance from float to int.
    train_data['compliance']=train_data['compliance'].astype('int')
    
    # Take only the first two characters of the violation code
    test_data['violation_code']=test_data['violation_code'].astype(str).str[0:2]
    train_data['violation_code']=train_data['violation_code'].astype(str).str[0:2]
    
    # Turn string features into one hot encoding
    string_features = ['agency_name','violation_code','disposition']
    train_data =  pd.get_dummies(train_data,columns = string_features,drop_first = True)
    test_data =  pd.get_dummies(test_data,columns = string_features,drop_first = True)
    
    #Remove columns from train data that do not appear in test data.
    train_cols_to_drop = [col for col in train_data.columns if col not in test_data.columns and col!='compliance']
    train_data.drop(train_cols_to_drop, axis=1,inplace=True)
    test_cols_to_drop = [col for col in test_data.columns if col not in train_data.columns]
    test_data.drop(test_cols_to_drop, axis=1,inplace=True)
    
    # Make train data 
    y = train_data['compliance']
    X = train_data.drop('compliance',axis = 1)
    X_test = test_data
    
    model = RandomForestRegressor(max_depth = 10, random_state=0).fit(X, y)
    ypred = model.predict(X_test)
    
    out = pd.Series(ypred, index=test_data.index)
    
    return out


    
#out = blight_model('readonly/')



/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [45]:
#out

ticket_id
284932    0.051443
285362    0.013649
285361    0.061071
285338    0.074068
285346    0.074068
285345    0.074068
285347    0.079996
285342    0.951632
285530    0.024216
284989    0.016924
285344    0.078696
285343    0.015044
285340    0.015044
285341    0.079996
285349    0.074068
285348    0.074068
284991    0.016924
285532    0.030871
285406    0.020781
285001    0.022575
285006    0.013586
285405    0.013649
285337    0.016557
285496    0.064151
285497    0.061467
285378    0.011061
285589    0.013330
285585    0.053062
285501    0.067317
285581    0.009615
            ...   
376367    0.013478
376366    0.030083
376362    0.170145
376363    0.159412
376365    0.013478
376364    0.030083
376228    0.040008
376265    0.031885
376286    0.089720
376320    0.038624
376314    0.037683
376327    0.283440
376385    0.278785
376435    0.954863
376370    0.951403
376434    0.073579
376459    0.073579
376478    0.008784
376473    0.040008
376484    0.022274
376482    0.010692
37

In [46]:
#np.histogram(out)

(array([50248,  3909,  2214,   460,   130,    82,   139,   310,   758,  2751]),
 array([ 0.00404334,  0.10363901,  0.20323467,  0.30283034,  0.402426  ,
         0.50202167,  0.60161734,  0.701213  ,  0.80080867,  0.90040433,  1.        ]))